In [ ]:
storageAccount="cookbookaadass2storage"
mountpoint = "/mnt/Gen2Source"
storageEndPoint ="abfss://rawdata@{}.dfs.core.windows.net/".format(storageAccount)
print ('Mount Point ='+mountpoint)

#ClientId, TenantId and Secret is for the Application(ADLSGen2App) was have created as part of this recipe
clientID ="xxxxx69-xx6e-4xxb-b6fa-a9sdad221210db"
tenantID =""
clientSecret =""
oauth2Endpoint = "https://login.microsoftonline.com/{}/oauth2/token".format(tenantID)


configs = {"fs.azure.account.auth.type": "OAuth",
           "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
           "fs.azure.account.oauth2.client.id": clientID,
           "fs.azure.account.oauth2.client.secret": clientSecret,
           "fs.azure.account.oauth2.client.endpoint": oauth2Endpoint}

try:
  dbutils.fs.mount(
  source = storageEndPoint,
  mount_point = mountpoint,
  extra_configs = configs)
except:
    print("Already mounted...."+mountpoint)


Mount Point =/mnt/Gen2Source
Already mounted..../mnt/Gen2Source

In [ ]:
display(dbutils.fs.ls("/mnt/Gen2Source/Customer/parquetFiles"))

path,name,size
dbfs:/mnt/Gen2Source/Customer/parquetFiles/_SUCCESS,_SUCCESS,0
dbfs:/mnt/Gen2Source/Customer/parquetFiles/_committed_1356276689186974440,_committed_1356276689186974440,1014
dbfs:/mnt/Gen2Source/Customer/parquetFiles/_started_1356276689186974440,_started_1356276689186974440,0
dbfs:/mnt/Gen2Source/Customer/parquetFiles/part-00000-tid-1356276689186974440-de7ea47e-e183-4819-acd1-0a633a5f75ba-62-1-c000.snappy.parquet,part-00000-tid-1356276689186974440-de7ea47e-e183-4819-acd1-0a633a5f75ba-62-1-c000.snappy.parquet,1250037
dbfs:/mnt/Gen2Source/Customer/parquetFiles/part-00001-tid-1356276689186974440-de7ea47e-e183-4819-acd1-0a633a5f75ba-63-1-c000.snappy.parquet,part-00001-tid-1356276689186974440-de7ea47e-e183-4819-acd1-0a633a5f75ba-63-1-c000.snappy.parquet,1244370
dbfs:/mnt/Gen2Source/Customer/parquetFiles/part-00002-tid-1356276689186974440-de7ea47e-e183-4819-acd1-0a633a5f75ba-64-1-c000.snappy.parquet,part-00002-tid-1356276689186974440-de7ea47e-e183-4819-acd1-0a633a5f75ba-64-1-c000.snappy.parquet,1246614
dbfs:/mnt/Gen2Source/Customer/parquetFiles/part-00003-tid-1356276689186974440-de7ea47e-e183-4819-acd1-0a633a5f75ba-65-1-c000.snappy.parquet,part-00003-tid-1356276689186974440-de7ea47e-e183-4819-acd1-0a633a5f75ba-65-1-c000.snappy.parquet,1246640
dbfs:/mnt/Gen2Source/Customer/parquetFiles/part-00004-tid-1356276689186974440-de7ea47e-e183-4819-acd1-0a633a5f75ba-66-1-c000.snappy.parquet,part-00004-tid-1356276689186974440-de7ea47e-e183-4819-acd1-0a633a5f75ba-66-1-c000.snappy.parquet,1252418
dbfs:/mnt/Gen2Source/Customer/parquetFiles/part-00005-tid-1356276689186974440-de7ea47e-e183-4819-acd1-0a633a5f75ba-67-1-c000.snappy.parquet,part-00005-tid-1356276689186974440-de7ea47e-e183-4819-acd1-0a633a5f75ba-67-1-c000.snappy.parquet,1249857
dbfs:/mnt/Gen2Source/Customer/parquetFiles/part-00006-tid-1356276689186974440-de7ea47e-e183-4819-acd1-0a633a5f75ba-68-1-c000.snappy.parquet,part-00006-tid-1356276689186974440-de7ea47e-e183-4819-acd1-0a633a5f75ba-68-1-c000.snappy.parquet,1248726


In [ ]:
db = "tpchdb"
 
spark.sql(f"CREATE DATABASE IF NOT EXISTS {db}")
spark.sql(f"USE {db}")
 
spark.sql("SET spark.databricks.delta.formatCheck.enabled = false")
spark.sql("SET spark.databricks.delta.properties.defaults.autoOptimize.optimizeWrite = true")

Out[11]: DataFrame[key: string, value: string]

In [ ]:
import random
from datetime import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import *
  

### Reading Customer parquet files from the mount point and writing to Delta tables.

In [ ]:
#Reading parquet files and adding a new column to the dataframe and writing to delta table
cust_path = "/mnt/Gen2Source/Customer/parquetFiles"
 
df_cust = (spark.read.format("parquet").load(cust_path)
      .withColumn("timestamp", current_timestamp()))
 
df_cust.write.format("delta").mode("overwrite").save("/mnt/Gen2Source/Customer/delta")

In [ ]:
%sql
-- Creating Delta table 
DROP TABLE IF EXISTS Customer;
CREATE TABLE Customer
USING delta
location "/mnt/Gen2Source/Customer/delta"

In [ ]:
%sql
describe formatted Customer

col_name,data_type,comment
C_CUSTKEY,int,
C_NAME,string,
C_ADDRESS,string,
C_NATIONKEY,smallint,
C_PHONE,string,
C_ACCTBAL,double,
C_MKTSEGMENT,string,
C_COMMENT,string,
timestamp,timestamp,
,,


In [ ]:
%sql
select * from Customer limit 10;

C_CUSTKEY,C_NAME,C_ADDRESS,C_NATIONKEY,C_PHONE,C_ACCTBAL,C_MKTSEGMENT,C_COMMENT,timestamp
82533,Customer#000082533,",ZbZT,39YiGQMmRrr5VndYyCm2c",0,10-647-805-7415,1307.12,AUTOMOBILE,hely regular packages haggle above the even requests,2021-04-11T09:26:25.235+0000
101275,Customer#000101275,Woi3b2ZaicPh ZSfu1EfXhE,0,10-988-842-1491,8346.85,FURNITURE,"gged, pending requests about the furiously ironic requests wake carefully ironic ideas. blithely bold h",2021-04-11T09:26:25.235+0000
55625,Customer#000055625,ylgY3fl2dbQEkvcDONDQPPgRGmqsXRQP99Wm0g,0,10-873-845-4881,1590.99,MACHINERY,lent packages are final instructions. alw,2021-04-11T09:26:25.235+0000
36375,Customer#000036375,INx32m88vH5u4gyJNO5FNt9 rJFFMM,0,10-123-638-4793,-833.84,BUILDING,s sleep carefully. even asymptotes hag,2021-04-11T09:26:25.235+0000
99703,Customer#000099703,qbER5guJSg4r8dqVEU,0,10-402-681-9759,8463.99,MACHINERY,ully final dugouts haggle quickly slyly even accounts.,2021-04-11T09:26:25.235+0000
62281,Customer#000062281,TIj SQWOQvlpzpHiJdu,0,10-279-547-5163,3751.96,BUILDING,blithely according to the slyly dogged requests. slyly final accoun,2021-04-11T09:26:25.235+0000
121751,Customer#000121751,Iw8Tvy733eX7GC2EvKI0,0,10-245-795-2362,6983.46,MACHINERY,slowly unusual courts eat final f,2021-04-11T09:26:25.235+0000
51674,Customer#000051674,XaQ6mDX9QrTuc7 C7,0,10-973-118-7525,1214.0,BUILDING,"o beans against the pinto beans use quickly after the slyly final epitaphs; express, regular theodolites sublate",2021-04-11T09:26:25.235+0000
35658,Customer#000035658,aXG4rmq1KG,0,10-271-855-5004,9529.97,FURNITURE,e slowly. carefully final accounts after the blithely silent ideas boost furiously carefully ironic dolphins! furiou,2021-04-11T09:26:25.235+0000
85448,Customer#000085448,8MlQOCflCXQ3RWXJ9AM7oonjIhw1xT110iDFU72f,0,10-338-309-9976,58.41,FURNITURE,sits across the dependencies dazzle furiously across the final foxes. quickly special deposits after the furiously f,2021-04-11T09:26:25.235+0000


In [ ]:
%fs ls /mnt/Gen2Source/Orders/parquetFiles/

path,name,size
dbfs:/mnt/Gen2Source/Orders/parquetFiles/part-00000-tid-413255148558796313-911f8f6b-4fad-417e-bae5-b9f0e06776f2-82-1-c000.snappy.parquet,part-00000-tid-413255148558796313-911f8f6b-4fad-417e-bae5-b9f0e06776f2-82-1-c000.snappy.parquet,867030
dbfs:/mnt/Gen2Source/Orders/parquetFiles/part-00001-tid-413255148558796313-911f8f6b-4fad-417e-bae5-b9f0e06776f2-83-1-c000.snappy.parquet,part-00001-tid-413255148558796313-911f8f6b-4fad-417e-bae5-b9f0e06776f2-83-1-c000.snappy.parquet,866929
dbfs:/mnt/Gen2Source/Orders/parquetFiles/part-00002-tid-413255148558796313-911f8f6b-4fad-417e-bae5-b9f0e06776f2-84-1-c000.snappy.parquet,part-00002-tid-413255148558796313-911f8f6b-4fad-417e-bae5-b9f0e06776f2-84-1-c000.snappy.parquet,866803
dbfs:/mnt/Gen2Source/Orders/parquetFiles/part-00003-tid-413255148558796313-911f8f6b-4fad-417e-bae5-b9f0e06776f2-85-1-c000.snappy.parquet,part-00003-tid-413255148558796313-911f8f6b-4fad-417e-bae5-b9f0e06776f2-85-1-c000.snappy.parquet,866795
dbfs:/mnt/Gen2Source/Orders/parquetFiles/part-00004-tid-413255148558796313-911f8f6b-4fad-417e-bae5-b9f0e06776f2-86-1-c000.snappy.parquet,part-00004-tid-413255148558796313-911f8f6b-4fad-417e-bae5-b9f0e06776f2-86-1-c000.snappy.parquet,866832
dbfs:/mnt/Gen2Source/Orders/parquetFiles/part-00005-tid-413255148558796313-911f8f6b-4fad-417e-bae5-b9f0e06776f2-87-1-c000.snappy.parquet,part-00005-tid-413255148558796313-911f8f6b-4fad-417e-bae5-b9f0e06776f2-87-1-c000.snappy.parquet,867069
dbfs:/mnt/Gen2Source/Orders/parquetFiles/part-00006-tid-413255148558796313-911f8f6b-4fad-417e-bae5-b9f0e06776f2-88-1-c000.snappy.parquet,part-00006-tid-413255148558796313-911f8f6b-4fad-417e-bae5-b9f0e06776f2-88-1-c000.snappy.parquet,866918
dbfs:/mnt/Gen2Source/Orders/parquetFiles/part-00007-tid-413255148558796313-911f8f6b-4fad-417e-bae5-b9f0e06776f2-89-1-c000.snappy.parquet,part-00007-tid-413255148558796313-911f8f6b-4fad-417e-bae5-b9f0e06776f2-89-1-c000.snappy.parquet,866973
dbfs:/mnt/Gen2Source/Orders/parquetFiles/part-00008-tid-413255148558796313-911f8f6b-4fad-417e-bae5-b9f0e06776f2-90-1-c000.snappy.parquet,part-00008-tid-413255148558796313-911f8f6b-4fad-417e-bae5-b9f0e06776f2-90-1-c000.snappy.parquet,867001
dbfs:/mnt/Gen2Source/Orders/parquetFiles/part-00009-tid-413255148558796313-911f8f6b-4fad-417e-bae5-b9f0e06776f2-91-1-c000.snappy.parquet,part-00009-tid-413255148558796313-911f8f6b-4fad-417e-bae5-b9f0e06776f2-91-1-c000.snappy.parquet,867085


In [ ]:
#Reading Orders parquet files and adding a new column to the dataframe and writing as delta format. Creating partition on year column.
ord_path = "/mnt/Gen2Source/Orders/parquetFiles"
 
df_ord = (spark.read.format("parquet").load(ord_path)
      .withColumn("timestamp", current_timestamp())
      .withColumn("O_OrderDateYear", year(col("O_OrderDate")))
     )

# df.printSchema()
 
df_ord.write.format("delta").partitionBy("O_OrderDateYear").mode("overwrite").save("/mnt/Gen2Source/Orders/delta")

In [ ]:
%sql
DROP TABLE IF EXISTS Orders;
CREATE TABLE Orders
USING delta
location "/mnt/Gen2Source/Orders/delta"

In [ ]:
%sql
describe formatted Orders

col_name,data_type,comment
O_ORDERKEY,int,
O_CUSTKEY,int,
O_ORDERSTATUS,string,
O_TOTALPRICE,"decimal(13,2)",
O_ORDERDATE,date,
O_ORDERPRIORITY,string,
O_CLERK,string,
O_SHIPPRIORITY,int,
O_COMMENT,string,
timestamp,timestamp,


In [ ]:
%sql
-- Getting total customer based on priority and total account balance by joing the delta tables
SELECT o.O_ORDERPRIORITY,count(o.O_CustKey) As TotalCustomer,Sum(c.C_AcctBal) As CustAcctBal
FROM Orders o 
INNER JOIN Customer c on o.O_CustKey=c.C_CustKey 
WHERE o.O_OrderDateYear>1997
GROUP BY o.O_OrderPriority
ORDER BY TotalCustomer DESC;



O_ORDERPRIORITY,TotalCustomer,CustAcctBal
5-LOW,23411,1.0472520520999993E8
3-MEDIUM,23227,1.0434247233999982E8
1-URGENT,23116,1.040591873499999E8
2-HIGH,23044,1.0384555805999993E8
4-NOT SPECIFIED,22999,1.0309230505999999E8


In [ ]:
# We can read the Delta files in Python using DeltaTable libraries as shown below 
deltaTable = DeltaTable.forPath(spark, "/mnt/Gen2Source/Orders/delta/")

In [ ]:
#Converting to DF and viewing dataframe contents
dt=deltaTable.toDF()
dt.show()

+----------+---------+-------------+------------+-----------+---------------+---------------+--------------+-------------------+--------------------+---------------+
O_ORDERKEY|O_CUSTKEY|O_ORDERSTATUS|O_TOTALPRICE|O_ORDERDATE|O_ORDERPRIORITY| O_CLERK|O_SHIPPRIORITY| O_COMMENT| timestamp|O_OrderDateYear|
+----------+---------+-------------+------------+-----------+---------------+---------------+--------------+-------------------+--------------------+---------------+
 1359427| 93106| F| 37461.23| 1992-06-22|4-NOT SPECIFIED|Clerk#000000918| 0|furiously special f|2021-04-12 17:10:...| 1992|
 900516| 126406| F| 75507.84| 1992-11-22| 2-HIGH|Clerk#000000668| 0|furiously special f|2021-04-12 17:10:...| 1992|
 1518148| 4870| F| 66636.43| 1992-06-18| 2-HIGH|Clerk#000000352| 0|furiously special f|2021-04-12 17:10:...| 1992|
 1224739| 21574| F| 117875.18| 1992-04-16| 1-URGENT|Clerk#000000986| 0|furiously special f|2021-04-12 17:10:...| 1992|
 1014887| 109319| F| 295234.75| 1992-09-26| 3-MEDIUM|Clerk#000000737| 0|furiously special f|2021-04-12 17:10:...| 1992|
 1233124| 26998| F| 150319.58| 1992-03-25| 3-MEDIUM|Clerk#000000724| 0|furiously special f|2021-04-12 17:10:...| 1992|
 808452| 143617| F| 217176.96| 1992-12-14| 5-LOW|Clerk#000000088| 0|furiously special f|2021-04-12 17:10:...| 1992|
 875143| 128503| F| 133716.36| 1992-09-27| 2-HIGH|Clerk#000000021| 0|furiously special f|2021-04-12 17:10:...| 1992|
 1170306| 35647| F| 174109.85| 1992-11-18|4-NOT SPECIFIED|Clerk#000000981| 0|furiously special f|2021-04-12 17:10:...| 1992|
 875973| 89810| F| 152861.25| 1992-02-09| 5-LOW|Clerk#000000065| 0|furiously special f|2021-04-12 17:10:...| 1992|
 1005761| 25060| F| 239564.41| 1992-02-28| 2-HIGH|Clerk#000000650| 0|furiously special f|2021-04-12 17:10:...| 1992|
 1163683| 84958| F| 232292.45| 1992-05-25|4-NOT SPECIFIED|Clerk#000000314| 0|furiously special f|2021-04-12 17:10:...| 1992|
 883364| 6083| F| 179796.95| 1992-10-03| 3-MEDIUM|Clerk#000000332| 0|furiously special f|2021-04-12 17:10:...| 1992|
 1048641| 16159| F| 31882.01| 1992-03-10|4-NOT SPECIFIED|Clerk#000000308| 0|furiously special f|2021-04-12 17:10:...| 1992|
 1465699| 141634| F| 126416.07| 1992-10-09| 2-HIGH|Clerk#000000761| 0|furiously special f|2021-04-12 17:10:...| 1992|
 920807| 118313| F| 303453.72| 1992-07-27| 1-URGENT|Clerk#000000556| 0|furiously special f|2021-04-12 17:10:...| 1992|
 1306084| 63431| F| 265863.22| 1992-05-24| 2-HIGH|Clerk#000000234| 0|furiously special f|2021-04-12 17:10:...| 1992|
 1235840| 63418| F| 26691.89| 1992-01-06| 5-LOW|Clerk#000000402| 0|furiously special f|2021-04-12 17:10:...| 1992|
 1165954| 119539| F| 28532.06| 1992-02-17| 2-HIGH|Clerk#000000466| 0|furiously special f|2021-04-12 17:10:...| 1992|
 858439| 132121| F| 271543.07| 1992-10-31|4-NOT SPECIFIED|Clerk#000000636| 0|furiously special f|2021-04-12 17:10:...| 1992|
+----------+---------+-------------+------------+-----------+---------------+---------------+--------------+-------------------+--------------------+---------------+
only showing top 20 rows

### DML Support in Delta Table.
  We can perform DML opreations like insert,delete and updates to the Delta table.

In [ ]:
# Reading new data which is in the folder newParquetFiles in the same orders folder in the /nnt/Gen2Source
# We will see how to perform merge to insert anf update the records in the delta table.

#Reading Orders parquet files and adding a new column to the dataframe and writing as delta format. Creating partition on year column.
new_ord_path = "/mnt/Gen2Source/Orders/newParquetFiles"
 
df_new_order = (spark.read.format("parquet").load(new_ord_path)
      .withColumn("timestamp", current_timestamp())
      .withColumn("O_OrderDateYear", year(col("O_OrderDate")))
     )

# df.printSchema()
 
df_new_order.write.format("delta").partitionBy("O_OrderDateYear").mode("overwrite").save("/mnt/Gen2Source/Orders/dailydata_delta")



In [ ]:
%sql
DROP TABLE IF EXISTS Orders_Daily;
CREATE TABLE Orders_Daily
USING delta
location "/mnt/Gen2Source/Orders/dailydata_delta"

In [ ]:
%sql
-- Merging into Orders from Orders_Daily table. If records matche on OrderKey then perform update eles insert
MERGE INTO Orders AS o
USING Orders_daily AS d
ON o.O_ORDERKEY = D.O_ORDERKEY
WHEN MATCHED THEN 
  UPDATE SET *
WHEN NOT MATCHED 
  THEN INSERT *;

In [ ]:
%sql
-- Deleting from Delta table
DELETE FROM Orders WHERE O_ORDERKEY=1359427

In [ ]:
%sql
SELECT * FROM ORDERS WHERE O_ORDERKEY=1359427

O_ORDERKEY,O_CUSTKEY,O_ORDERSTATUS,O_TOTALPRICE,O_ORDERDATE,O_ORDERPRIORITY,O_CLERK,O_SHIPPRIORITY,O_COMMENT,timestamp,O_OrderDateYear


In [ ]:
%fs ls /mnt/Gen2Source/Orders/ParquetOutputFormatted

path,name,size
dbfs:/mnt/Gen2Source/Orders/ParquetOutputFormatted/_SUCCESS,_SUCCESS,0
dbfs:/mnt/Gen2Source/Orders/ParquetOutputFormatted/_committed_5983648424092008672,_committed_5983648424092008672,824
dbfs:/mnt/Gen2Source/Orders/ParquetOutputFormatted/_started_5983648424092008672,_started_5983648424092008672,0
dbfs:/mnt/Gen2Source/Orders/ParquetOutputFormatted/part-00000-tid-5983648424092008672-105fe66c-3ac1-4a96-a8e8-4407bbb85196-210-1-c000.snappy.parquet,part-00000-tid-5983648424092008672-105fe66c-3ac1-4a96-a8e8-4407bbb85196-210-1-c000.snappy.parquet,3375529
dbfs:/mnt/Gen2Source/Orders/ParquetOutputFormatted/part-00001-tid-5983648424092008672-105fe66c-3ac1-4a96-a8e8-4407bbb85196-211-1-c000.snappy.parquet,part-00001-tid-5983648424092008672-105fe66c-3ac1-4a96-a8e8-4407bbb85196-211-1-c000.snappy.parquet,3375655
dbfs:/mnt/Gen2Source/Orders/ParquetOutputFormatted/part-00002-tid-5983648424092008672-105fe66c-3ac1-4a96-a8e8-4407bbb85196-212-1-c000.snappy.parquet,part-00002-tid-5983648424092008672-105fe66c-3ac1-4a96-a8e8-4407bbb85196-212-1-c000.snappy.parquet,3376066
dbfs:/mnt/Gen2Source/Orders/ParquetOutputFormatted/part-00003-tid-5983648424092008672-105fe66c-3ac1-4a96-a8e8-4407bbb85196-213-1-c000.snappy.parquet,part-00003-tid-5983648424092008672-105fe66c-3ac1-4a96-a8e8-4407bbb85196-213-1-c000.snappy.parquet,3374087
dbfs:/mnt/Gen2Source/Orders/ParquetOutputFormatted/part-00004-tid-5983648424092008672-105fe66c-3ac1-4a96-a8e8-4407bbb85196-214-1-c000.snappy.parquet,part-00004-tid-5983648424092008672-105fe66c-3ac1-4a96-a8e8-4407bbb85196-214-1-c000.snappy.parquet,3373949
dbfs:/mnt/Gen2Source/Orders/ParquetOutputFormatted/part-00005-tid-5983648424092008672-105fe66c-3ac1-4a96-a8e8-4407bbb85196-215-1-c000.snappy.parquet,part-00005-tid-5983648424092008672-105fe66c-3ac1-4a96-a8e8-4407bbb85196-215-1-c000.snappy.parquet,3374601
dbfs:/mnt/Gen2Source/Orders/ParquetOutputFormatted/part-00006-tid-5983648424092008672-105fe66c-3ac1-4a96-a8e8-4407bbb85196-216-1-c000.snappy.parquet,part-00006-tid-5983648424092008672-105fe66c-3ac1-4a96-a8e8-4407bbb85196-216-1-c000.snappy.parquet,3374168


In [ ]:
%sql CONVERT TO DELTA parquet.`dbfs:/mnt/Gen2Source/Orders/ParquetOutputFormatted`

In [ ]:
%sql
CREATE TABLE Orders_Test
USING DELTA
LOCATION "/mnt/Gen2Source/Orders/ParquetOutputFormatted"

In [ ]:
%sql
describe formatted Orders_Test

col_name,data_type,comment
O_ORDERKEY,int,
O_CUSTKEY,int,
O_ORDERSTATUS,string,
O_TOTALPRICE,"decimal(13,2)",
O_ORDERDATE,date,
O_ORDERPRIORITY,string,
O_CLERK,string,
O_SHIPPRIORITY,int,
O_COMMENT,string,
,,


In [ ]:
#dbutils.fs.unmount("/mnt/Gen2Source")

/mnt/Gen2Source has been unmounted.
Out[3]: True